In [328]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader,random_split
from torch.optim.lr_scheduler import StepLR,ReduceLROnPlateau
import torchmetrics.functional as metrics
import os
import shutil
from torchvision import transforms
from PIL import Image

In [295]:


folder_path = '../train/train'
target_data = []
img_data = []
for encoding_label,label in enumerate(os.listdir(folder_path)):
    label_path = os.path.join(folder_path, label)
    if os.path.isdir(label_path):  # 디렉토리인 경우에만 진입
        for img in os.listdir(folder_path+'/'+label):
            image_path = os.path.join(folder_path,label,img)
            if os.path.isfile(image_path):  # 파일인 경우에만 진입
                with open(image_path, 'rb') as file:
                    image = Image.open(file)
                    width, height = image.size
                    if width == 48 and height == 48:
                        image_array = np.array(image)
                        target_data.append(encoding_label)
                        img_data.append(image_array)

In [296]:
os.listdir(folder_path)

['happy',
 '.DS_Store',
 'sad',
 'fear',
 'surprise',
 'neutral',
 'angry',
 'disgust']

In [297]:
len(img_data), len(target_data)

(28709, 28709)

In [298]:
type(img_data), type(target_data)

(list, list)

In [299]:
img_data[0].shape

(48, 48)

In [300]:
a = np.array(img_data)
b = a/255.
c = b.reshape(-1,48*48)
c

array([[0.42352941, 0.3254902 , 0.24705882, ..., 0.45490196, 0.47843137,
        0.47843137],
       [0.5372549 , 0.55686275, 0.62352941, ..., 0.74901961, 0.75294118,
        0.75294118],
       [0.43529412, 0.58039216, 0.60784314, ..., 0.22352941, 0.21960784,
        0.23529412],
       ...,
       [0.17647059, 0.17254902, 0.16862745, ..., 0.68627451, 0.77647059,
        0.8627451 ],
       [0.23921569, 0.15294118, 0.23921569, ..., 0.63921569, 0.63137255,
        0.62745098],
       [1.        , 0.98431373, 1.        , ..., 0.69803922, 0.70588235,
        0.75686275]])

In [330]:
c.shape

(28709, 2304)

In [302]:
norm_target = np.array(target_data)

In [303]:
# norm_target = torch.from_numpy(norm_target)

In [304]:
norm_feature = np.array(img_data)/255.


In [305]:
norm_feature = torch.tensor(c, dtype=torch.float32)
norm_target = torch.tensor(norm_target)

In [306]:
# norm_feature = torch.from_numpy(norm_feature)

In [307]:
pd.Series(norm_target).value_counts()

0    7215
5    4965
2    4830
3    4097
6    3995
4    3171
7     436
Name: count, dtype: int64

In [308]:
type(pd.Series(norm_target)[0])

numpy.int64

In [309]:
norm_targetReplace = pd.Series(norm_target).replace({0:3, 5:4, 2:5, 3:2, 6:0, 4:6, 7:1})
norm_targetReplace.value_counts()

3    7215
4    4965
5    4830
2    4097
0    3995
6    3171
1     436
Name: count, dtype: int64

In [310]:
norm_feature.shape, norm_target.shape

(torch.Size([28709, 2304]), torch.Size([28709]))

In [311]:
class DLDataset(Dataset):
    def __init__(self, x_data, y_data):
        super().__init__()
        self.feature=torch.FloatTensor(x_data)
        self.target=torch.LongTensor(y_data)

    def __len__(self):
        return self.target.shape[0]
    
    def __getitem__(self, index):
        return self.feature[index], self.target[index]

In [312]:
all_dataset= DLDataset(norm_feature, norm_target)

In [313]:
from torch.utils.data import random_split

seed = torch.Generator().manual_seed(42)
trainDS, validDS = random_split(all_dataset, [0.8,0.2], generator=seed)

In [314]:
folder_path = '../test/test'
test_target_data = []
test_img_data = []
for encoding_label,label in enumerate(os.listdir(folder_path)):
    label_path = os.path.join(folder_path, label)
    if os.path.isdir(label_path):  # 디렉토리인 경우에만 진입
        for img in os.listdir(folder_path+'/'+label):
            image_path = os.path.join(folder_path,label,img)
            if os.path.isfile(image_path):  # 파일인 경우에만 진입
                with open(image_path, 'rb') as file:
                    test_image = Image.open(file)
                    test_image_array = np.array(image)
                    test_target_data.append(encoding_label)
                    test_img_data.append(image_array)

In [315]:
pd.Series(test_target_data).unique()
pd.Series(test_target_data).value_counts()

0    1774
1    1247
4    1233
2    1024
5     958
3     831
6     111
Name: count, dtype: int64

In [316]:
norm_test_target = np.array(test_target_data)
norm_test_feature = np.array(test_img_data)/255.


In [317]:

testDS= DLDataset(norm_test_feature, norm_test_target)

In [318]:
batch = 30
trainDL = DataLoader(trainDS, batch_size=batch, drop_last=True)
validDL = DataLoader(validDS, batch_size=batch, drop_last=True)
testDL = DataLoader(testDS, batch_size=batch, drop_last=True)


In [319]:
len(trainDS), len(validDS), len(testDL)


(22968, 5741, 239)

In [320]:
norm_target

tensor([0, 0, 0,  ..., 7, 7, 7])

In [321]:
import torch.nn as nn

class Model1(nn.Module):
    
    # 모델 구조 설정
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.input_layer = nn.Linear(in_dim, 100)
        self.relu = nn.ReLU()  # 0보다 작거나 같으면 0 / 0보다 크면 1
        self.hidden_layer = nn.Linear(100, 27) 
        self.output_layer = nn.Linear(27, out_dim) 
        
    # 순방향 학습 진행 콜백 함수
    def forward(self, x): # 입력데이터 : x
        y = self.input_layer(x)   # W1*X1 + W2*X2 + ..Wm*Xm +b 100개 반환
        
        y = self.relu(y)          # relu() 결과 100개 반환  
        
        y = self.hidden_layer(y)   # W1*X1 + W2*X2 + ..Wm*Xm +b 27개 반환
        
        y = self.relu(y)          # relu() 결과 27개 반환
        
        y = self.output_layer(y)   # W1*X1 + W2*X2 + ..Wm*Xm +b 3개 반환
        
        return y

In [322]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

EPOCHS = 50

IN_DIM = all_dataset.feature.shape[1]
OUT_DIM = len(torch.unique(all_dataset.target))  # or len(np.unique(targetNP) or tragetDF.nunique()

model = Model1(IN_DIM, OUT_DIM).to(DEVICE)

LOSS_FUNCTION = nn.CrossEntropyLoss().to(DEVICE)

import torch.optim as optim

OPTIMIZER = optim.Adam(model.parameters())

In [323]:
# norm_feature = torch.DoubleTensor(norm_feature)b

In [335]:
def training(dataLoader):
    # 학습모드 => 정규화, 경사하강법, 드랍아웃 등의 기능 활성화
    model.train() 
    
    # 배치크기만큼 학습진행 및 저장
    train_loss = 0
    train_acc = 0
    for cnt, (feature, target) in enumerate(dataLoader):
        feature, target = feature.to(DEVICE), target.to(DEVICE)
        # 학습
        pre_target = model(norm_feature)

        # 손실계산
        loss = LOSS_FUNCTION(pre_target,target)
        train_loss += loss.item()
        
        # 정확도 
        acc = metrics.accuracy(pre_target, target, task = 'multiclass', num_classes=len(torch.unique(all_dataset.target)))
        train_acc += acc.item()
        
        # W, b 업데이트 
        OPTIMIZER.zero_grad()
        loss.backward()
        OPTIMIZER.step()
        # 배치 사이즈 단위 학습 진행 메시지 출력
        # print(f'[{cnt}] : [Train batch loss] ==> {loss}')

    # 에포크 단위 학습 진행 메시지 출력
    print(f'[Train loss] ==> {train_loss/ len(dataLoader)}    [Train Accuracy] ==> {train_acc / len(dataLoader)}')
    
    return train_loss/ len(dataLoader), train_acc / len(dataLoader)
    
# train_loss, train_acc = training(trainDL)

In [336]:
def testing(dataLoader):
    # 학습모드 => 정규화, 경사하강법, 드랍아웃 등의 기능 비활성화
    model.eval()
    
    with torch.no_grad():
        # 배치크기만큼 학습진행
        val_loss = 0    
        val_acc = 0
        for cnt, (feature, target) in enumerate(dataLoader):
            feature, target = feature.to(DEVICE), target.to(DEVICE)
            # 학습
            pre_target = model(norm_feature)
    
            # 손실계산
            loss = LOSS_FUNCTION(pre_target,target)
            val_loss += loss.item()
            
            # 정확도 
            acc = metrics.accuracy(pre_target, target, task = 'multiclass', num_classes=len(torch.unique(all_dataset.target)))
            val_acc += acc.item()
              # 배치 사이즈 단위 학습 진행 메시지 출력
            # print(f'[{cnt}] : [Train batch loss] ==> {loss}')

    # 에포크 단위 학습 진행 메시지 출력
    print(f'[Valid loss] ==> {val_loss/ len(dataLoader)}    [Valid Accuracy] ==> {val_acc / len(dataLoader)}')
    
    return val_loss/ len(dataLoader), val_acc / len(dataLoader)
    
# val_loss, val_acc = testing(validDL)

In [337]:
SCHEDULER = ReduceLROnPlateau(OPTIMIZER, mode = 'min', patience = 3, verbose = True)

/Users/sml/anaconda3/envs/Torch_PY38/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [338]:
EPOCHS = 30
min_loss = 100.0  # 초기 최소 손실 설정
cnt = 0
for eps in range(EPOCHS):
    print(f'[{eps+1}/{EPOCHS}]')
    # 학습
    train_loss, train_acc = training(trainDL)

    # 검증
    val_loss, val_acc = testing(validDL)

    # 최소 손실 업데이트
    if val_loss < min_loss:
        min_loss = val_loss
        cnt = 0
    else:
        cnt+=1

    # 조기 종료 기능 => 조건 : val_loss가 지정된 횟수 이상 개선이 안되면 학습 종료
    if SCHEDULER.num_bad_epochs >= SCHEDULER.patience or cnt >= 2:
        print(f"Early stopping at epoch {eps}")
        break

[1/30]


ValueError: Expected input batch_size (28709) to match target batch_size (30).